## Install requirements

In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleOCR
!pip install paddlepaddle-gpu pyclipper attrdict -qqq
%cd PaddleOCR
!pip install -e .

# Import library

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import yaml

# Config

In [ ]:
%cd PaddleOCR

In [ ]:
%ls

In [ ]:
%cp /kaggle/input/cva-trainning/PaddleOCR/train_data -r /kaggle/working/PaddleOCR

In [ ]:
%cp /kaggle/input/cva-trainning/PaddleOCR/pretrain_models -r /kaggle/working/PaddleOCR

In [ ]:
content = """
Global:
  use_gpu: true
  epoch_num: 500
  log_smooth_window: 20
  print_batch_step: 2
  save_model_dir: ./output/db_r50_vd_tt/
  save_epoch_step: 1
  # evaluation is run every 5000 iterations after the 4000th iteration
  eval_batch_step: [4000, 5000]
  cal_metric_during_train: False
  pretrained_model: ./pretrain_models/ResNet18_vd_pretrained.pdparams
  checkpoints: 
  save_inference_dir: ./output/db_r50_vd_tt/
  use_visualdl: False
  infer_img: ./train_data/total_text/test_images/img1.jpg
  save_res_path: ./output/db_r50_vd_tt/predicts_db_tt.txt
  distributed: true

Architecture:
  model_type: det
  algorithm: DB
  Transform:
  Backbone:
    name: ResNet
    layers: 50
  Neck:
    name: DBFPN
    out_channels: 256
  Head:
    name: DBHead
    k: 50
    
Loss:
  name: DBLoss
  balance_loss: True
  main_loss_type: DiceLoss
  alpha: 5
  beta: 10
  ohem_ratio: 3

  
Optimizer:
  name: Adam
  beta1: 0.9
  beta2: 0.999
  lr:
  #  name: Cosine
    learning_rate: 0.001
  #  warmup_epoch: 0
  regularizer:
    name: 'L2'
    factor: 0

PostProcess:
  name: DBPostProcess
  thresh: 0.3
  box_thresh: 0.5
  max_candidates: 1000
  unclip_ratio: 1.5
  
Metric:
  name: DetMetric
  main_indicator: hmean

Train:
  dataset:
    name: SimpleDataSet
    data_dir: ./train_data/total_text/
    label_file_list: 
        - ./train_data/total_text/train_totaltext_label.txt
        - ./train_data/total_text/train_icdar2015_label.txt

    ratio_list: [1.0, 1.0]
    transforms:
      - DecodeImage: # load image
          img_mode: BGR
          channel_first: False
      - DetLabelEncode: # Class handling label
      - IaaAugment:
          augmenter_args:
            - { 'type': Fliplr, 'args': { 'p': 0.5 } }
            - { 'type': Affine, 'args': { 'rotate': [-10, 10] } }
            - { 'type': Resize, 'args': { 'size': [0.5, 3] } }
      - EastRandomCropData:
          size: [640, 640]
          max_tries: 50
          keep_ratio: true
      - MakeBorderMap:
          shrink_ratio: 0.4
          thresh_min: 0.3
          thresh_max: 0.7
      - MakeShrinkMap:
          shrink_ratio: 0.4
          min_text_size: 8
      - NormalizeImage:
          scale: 1./255.
          mean: [0.485, 0.456, 0.406]
          std: [0.229, 0.224, 0.225]
          order: 'hwc'
      - ToCHWImage:
      - KeepKeys:
          keep_keys: ['image', 'threshold_map', 'threshold_mask', 'shrink_map', 'shrink_mask'] # the order of the dataloader list

  loader:
    shuffle: True
    drop_last: False
    batch_size_per_card: 4
    num_workers: 8

Eval:
  dataset:
    name: SimpleDataSet
    data_dir: ./train_data/total_text/
    label_file_list: 
        - ./train_data/total_text/test_totaltext_label.txt
        - ./train_data/total_text/test_icdar2015_label.txt
    ratio_list: [1.0, 1.0]
    transforms:
      - DecodeImage: # load image
          img_mode: BGR
          channel_first: False
      - DetLabelEncode: # Class handling label
      - DetResizeForTest:
          image_shape: [736, 1280]
      - NormalizeImage:
          scale: 1./255.
          mean: [0.485, 0.456, 0.406]
          std: [0.229, 0.224, 0.225]
          order: 'hwc'
      - ToCHWImage:
      - KeepKeys:
          keep_keys: ['image', 'shape', 'polys', 'ignore_tags']

  loader:
    shuffle: False
    drop_last: False
    batch_size_per_card: 1 # must be 1
    num_workers: 8

"""

In [ ]:
# Specify the path where you want to save the YAML file
file_path = "/kaggle/working/PaddleOCR/configs/det/det_r50_db_totaltext.yml"

# Open the file in write mode and save the content
with open(file_path, "w") as file:
    file.write(content)

print(f"Configuration saved to {file_path}")

In [ ]:
os.rename("/kaggle/working/PaddleOCR/train_data/total_text/icdar_c4_test_imgs", "/kaggle/working/PaddleOCR/train_data/total_text/ch4_test_images")

In [ ]:
!cd train_data && ls && cd total_text && ls

# Run training

In [ ]:
!python3 -m paddle.distributed.launch --gpus '0,1' tools/train.py -c configs/det/det_r50_db_totaltext.yml -o Optimizer.base_lr=0.001 Global.epoch_num=500